In [1]:
import pandas as pd
import os

In [2]:
###Agregar poblacion
df_poblacion = pd.read_csv("data/osb_demografia-poblacion19072022.csv",sep = ';',encoding='latin-1')
df_poblacion.rename(columns ={"A¤o":"year","Cod_Loc":"Codigo localidad","Poblaci¢n": "Poblacion"},inplace = True)
df_pob = df_poblacion.groupby(["year","Codigo localidad"]).sum().reset_index()
df_pob = df_pob[["year","Codigo localidad","Poblacion"]]
df_pob


,year,Codigo localidad,Poblacion
0,2005,0,6710910
1,2005,1,415099
2,2005,2,121307
3,2005,3,96046
4,2005,4,400187
...,...,...,...
646,2035,16,264460
647,2035,17,21601
648,2035,18,412281
649,2035,19,731503


In [3]:
##MUERTES POR LOCALIDADES
ruta_localidad = os.getcwd()+'\\data\\osb_demografia-causasmortalidad.csv'
df_localidad = pd.read_csv(ruta_localidad,sep=';',encoding='latin-1')
df_localidad.drop(['MORTALIDAD_DESCRIPCION_105',
       'MORTALIDAD_DESCRIPCION_667', 'MORTALIDAD_CONDICION_CRONICA'], axis = 1, inplace = True)
df_localidad.rename(columns = {'ANIO': 'ANO', 'EDAD_GRUPO':'Grupo_Edad_Causa' ,
                                     'MORTALIDAD_CODIGO_CIE10':'C_BAS1',
                              'SEXO':'Sexo', 'LOCALIDAD_CODIGO' :'Código localidad',
                              'LOCALIDAD_NOMBRE': 'Localidad', 
                               'SEGURIDAD_SOCIAL_REGIMEN':'Régimen de seguridad social',
                              'MORTALIDAD_DESCRIPCION_CIE10':'Descripción causa básica de muerte'},inplace =True)

In [4]:
###Cargar lista de muertes por causas cardiometabolicas
ruta_causas = os.getcwd()+'\\data\\CausasCardioMetabolicas.xlsx'
df_causas = pd.read_excel(ruta_causas)
df_causas =  df_causas[df_causas['Cardio-metabolicas']==1].copy()
df_causas

,C_BAS1,Cardio-metabolicas
394,E101,1
395,E102,1
396,E105,1
397,E106,1
398,E107,1
...,...,...
711,I859,1
712,I864,1
713,I871,1
714,I872,1


In [5]:
df_muertes0 = pd.merge(df_localidad,df_causas, how = 'right', on = 'C_BAS1')
print(len(df_localidad))
print(len(df_muertes0))

539801
159854


In [6]:
df_muertes = df_muertes0.groupby(["ANO","Código localidad"]).count().reset_index()
df_muertes = df_muertes[["ANO","Código localidad","Sexo"]]
df_muertes.rename(columns={"Código localidad":"Codigo localidad","Sexo":"MuertesxPM25",
                          "ANO":"year"},inplace =True)
df_muertes

,year,Codigo localidad,MuertesxPM25
0,2005,1.0,489
1,2005,2.0,209
2,2005,3.0,199
3,2005,4.0,519
4,2005,5.0,242
...,...,...,...
335,2021,16.0,585
336,2021,17.0,60
337,2021,18.0,779
338,2021,19.0,1060


In [7]:
df_HR = pd.read_excel("../4. Curva C-R/salidas/04-HR_Localidad.xlsx")
df_HR.rename(columns={"Codigo.localidad":"Codigo localidad"},inplace =True)
df_HR

,Codigo localidad,year,pm25_pred,pm25_pred_prom,HR,HR_l,HR_u
0,1,2008,12.607740,12.607740,NaN,NaN,NaN
1,1,2009,12.381543,12.494642,NaN,NaN,NaN
2,1,2010,12.818953,12.602745,NaN,NaN,NaN
3,1,2011,11.929228,12.434366,NaN,NaN,NaN
4,1,2012,12.229965,12.393486,NaN,NaN,NaN
...,...,...,...,...,...,...,...
261,19,2017,26.473285,28.118326,1.231016,1.210198,1.252193
262,19,2018,25.605810,27.889916,1.221777,1.201823,1.242063
263,19,2019,26.466771,27.771320,1.230949,1.210137,1.252119
264,19,2020,26.013815,27.636128,1.226190,1.205833,1.246891


In [8]:
df_HR[['HR', 'HR_l', 'HR_u']] = df_HR[['HR', 'HR_l', 'HR_u']].fillna(1)

In [9]:
df_HR[['HR', 'HR_l', 'HR_u']] = df_HR[['HR', 'HR_l', 'HR_u']].applymap(lambda x: 1 if x < 1 else x)

In [10]:
df_mortalidad0 = pd.merge(df_HR,df_muertes,how='left',on = ['year','Codigo localidad'])
df_mortalidad = pd.merge(df_mortalidad0,df_pob,how='left',on = ['year','Codigo localidad'])
df_mortalidad

,Codigo localidad,year,pm25_pred,pm25_pred_prom,HR,HR_l,HR_u,MuertesxPM25,Poblacion
0,1,2008,12.607740,12.607740,1.000000,1.000000,1.000000,531,465244
1,1,2009,12.381543,12.494642,1.000000,1.000000,1.000000,533,477857
2,1,2010,12.818953,12.602745,1.000000,1.000000,1.000000,582,488850
3,1,2011,11.929228,12.434366,1.000000,1.000000,1.000000,612,498133
4,1,2012,12.229965,12.393486,1.000000,1.000000,1.000000,652,505657
...,...,...,...,...,...,...,...,...,...
261,19,2017,26.473285,28.118326,1.231016,1.210198,1.252193,682,606799
262,19,2018,25.605810,27.889916,1.221777,1.201823,1.242063,767,613127
263,19,2019,26.466771,27.771320,1.230949,1.210137,1.252119,753,628670
264,19,2020,26.013815,27.636128,1.226190,1.205833,1.246891,721,641306


In [11]:
df_mortalidad['muertes_estimadas'] = (df_mortalidad['HR']-1)*(df_mortalidad['MuertesxPM25'])/df_mortalidad['HR']
df_mortalidad

,Codigo localidad,year,pm25_pred,pm25_pred_prom,HR,HR_l,HR_u,MuertesxPM25,Poblacion,muertes_estimadas
0,1,2008,12.607740,12.607740,1.000000,1.000000,1.000000,531,465244,0.000000
1,1,2009,12.381543,12.494642,1.000000,1.000000,1.000000,533,477857,0.000000
2,1,2010,12.818953,12.602745,1.000000,1.000000,1.000000,582,488850,0.000000
3,1,2011,11.929228,12.434366,1.000000,1.000000,1.000000,612,498133,0.000000
4,1,2012,12.229965,12.393486,1.000000,1.000000,1.000000,652,505657,0.000000
...,...,...,...,...,...,...,...,...,...,...
261,19,2017,26.473285,28.118326,1.231016,1.210198,1.252193,682,606799,127.986249
262,19,2018,25.605810,27.889916,1.221777,1.201823,1.242063,767,613127,139.226059
263,19,2019,26.466771,27.771320,1.230949,1.210137,1.252119,753,628670,141.276837
264,19,2020,26.013815,27.636128,1.226190,1.205833,1.246891,721,641306,132.999860


In [12]:
df_mortalidad['muertes_estimadas_l'] = (df_mortalidad['HR_l']-1)*(df_mortalidad['MuertesxPM25'])/df_mortalidad['HR_l']
df_mortalidad['muertes_estimadas_u'] = (df_mortalidad['HR_u']-1)*(df_mortalidad['MuertesxPM25'])/df_mortalidad['HR_u']


In [13]:
df_mortalidad['%muertes_estimadas'] = df_mortalidad['muertes_estimadas']/df_mortalidad['MuertesxPM25']
df_mortalidad['%muertes_estimadas_l'] = df_mortalidad['muertes_estimadas_l']/df_mortalidad['MuertesxPM25']
df_mortalidad['%muertes_estimadas_u'] = df_mortalidad['muertes_estimadas_u']/df_mortalidad['MuertesxPM25']

In [14]:
localidades = pd.read_excel('data/Localidades.xlsx')
df_mortalidad1 = pd.merge(df_mortalidad,localidades, how='left',on='Codigo localidad')
df_mortalidad1

,Codigo localidad,year,pm25_pred,pm25_pred_prom,HR,HR_l,HR_u,MuertesxPM25,Poblacion,muertes_estimadas,muertes_estimadas_l,muertes_estimadas_u,%muertes_estimadas,%muertes_estimadas_l,%muertes_estimadas_u,Localidad
0,1,2008,12.607740,12.607740,1.000000,1.000000,1.000000,531,465244,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Usaquén
1,1,2009,12.381543,12.494642,1.000000,1.000000,1.000000,533,477857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Usaquén
2,1,2010,12.818953,12.602745,1.000000,1.000000,1.000000,582,488850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Usaquén
3,1,2011,11.929228,12.434366,1.000000,1.000000,1.000000,612,498133,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Usaquén
4,1,2012,12.229965,12.393486,1.000000,1.000000,1.000000,652,505657,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Usaquén
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,19,2017,26.473285,28.118326,1.231016,1.210198,1.252193,682,606799,127.986249,118.455685,137.355634,0.187663,0.173689,0.201401,Ciudad Bolívar
262,19,2018,25.605810,27.889916,1.221777,1.201823,1.242063,767,613127,139.226059,128.802856,149.479028,0.181520,0.167931,0.194888,Ciudad Bolívar
263,19,2019,26.466771,27.771320,1.230949,1.210137,1.252119,753,628670,141.276837,130.756318,151.619481,0.187619,0.173647,0.201354,Ciudad Bolívar
264,19,2020,26.013815,27.636128,1.226190,1.205833,1.246891,721,641306,132.999860,123.073170,142.761749,0.184466,0.170698,0.198005,Ciudad Bolívar


In [15]:
df_mortalidad1.to_excel('salidas/estimacion_muertes.xlsx',index =False)